In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoImageProcessor, ResNetModel
import torch
from datasets import load_dataset, Image
from sklearn.metrics import precision_recall_fscore_support
# from PIL import Image

In [ ]:
# from PIL import Image
# # Initialize image processor and ResNet model
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, otherwise use CPU
# image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
# model = ResNetModel.from_pretrained("microsoft/resnet-50").to(device)
# def get_image_vectors(dataset, split):
#     # Initialize list to store outputs
#     outputs_list = []

#     # Iterate through all images in the dataset
#     for i in range(len(dataset[split]['img'])):

#         # Open image using PIL
#         image = Image.open(datafolder+dataset[split]['img'][i]['path'])
        
#         # Check image format and convert if necessary
#         if image.mode != "RGB":
#             image = image.convert("RGB")

#         # Check image size and resize if necessary
#         if image.size != (224, 224):
#             image = image.resize((224, 224))
            
#         # Preprocess image using the image processor
#         inputs = image_processor(image, return_tensors="pt").to(device)

#         # Pass image through the ResNet model
#         with torch.no_grad():
#             outputs = model(**inputs)

#         # Append the last hidden state to the outputs list
#         outputs_list.append(outputs.last_hidden_state.to(device))
#     return outputs_list
#         # outputs_list.append(outputs.last_hidden_state.numpy().flatten().tolist())        

In [ ]:
# train_img = get_image_vectors(dataset, 'train')
# dev_img = get_image_vectors(dataset, 'validation')
# test_img = get_image_vectors(dataset, 'test')

# torch.save(train_img, 'train_img_tensors.pt')
# torch.save(dev_img, 'dev_img_tensors.pt')
# torch.save(test_img, 'test_img_tensors.pt')

In [ ]:
# importing the data
datafolder = '../../data/hateful_memes/'
train = datafolder+'train_with_features.csv'
test = datafolder+'test_with_features.csv'
dev = datafolder+'dev_with_features.csv'

In [ ]:
# Load the train, dev, and test datasets
train_data = load_dataset('csv', data_files=train, num_proc=8).cast_column("img", Image(decode=False))
dev_data = load_dataset('csv', data_files=dev, num_proc=8).cast_column("img", Image(decode=False))
test_data = load_dataset('csv', data_files=test, num_proc=8).cast_column("img", Image(decode=False))

Found cached dataset csv (C:/Users/Hisha/.cache/huggingface/datasets/csv/default-d08f24387b6055f1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset csv (C:/Users/Hisha/.cache/huggingface/datasets/csv/default-ad166f4d6492fd09/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset csv (C:/Users/Hisha/.cache/huggingface/datasets/csv/default-83fc309eac32ce13/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
train_img = torch.stack(torch.load('train_img_tensors.pt'))
dev_img = torch.stack(torch.load('dev_img_tensors.pt'))
test_img = torch.stack(torch.load('test_img_tensors.pt'))

In [7]:
Y_train = np.asarray(train_data['train']['label'])
Y_dev = np.asarray(dev_data['train']['label'])
Y_test = np.asarray(test_data['train']['label'])

In [26]:
# Create datasets
train_dataset = TensorDataset(train_img, torch.Tensor(Y_train))
dev_dataset = TensorDataset(dev_img, torch.Tensor(Y_dev))
test_dataset = TensorDataset(test_img, torch.Tensor(Y_test))

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# CNN

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, otherwise use CPU
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(2048, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.fc1 = nn.Linear(128 * 3 * 3, 512)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 2048, 7, 7)  # Reshape input tensor
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = x.view(-1, 128 * 3 * 3)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create model and optimizer
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [79]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    train_correct = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        if data.shape[0] != batch_size:
            continue
        optimizer.zero_grad()
        output = model(data)
        target = target.to(torch.int64)
        # output = output.to(torch.short)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc = 100. * train_correct / len(train_loader.dataset)
    return train_loss, train_acc

def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    test_correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            target = target.to(torch.int64)
            test_loss += criterion(output, target).item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)
            test_correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc = 100. * test_correct / len(test_loader.dataset)
    return test_loss, test_acc

In [101]:
# Define hyperparameters
num_epochs = 20
criterion = nn.CrossEntropyLoss()

# Train the model
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    dev_loss, dev_acc = test(model, dev_loader, criterion, device)
    print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Dev Loss: {dev_loss:.4f}, Dev Acc: {dev_acc:.2f}%')

Epoch 1: Train Loss: 0.0361, Train Acc: 98.81%, Dev Loss: 3.1533, Dev Acc: 47.40%
Epoch 2: Train Loss: 0.0242, Train Acc: 99.13%, Dev Loss: 3.8909, Dev Acc: 46.40%
Epoch 3: Train Loss: 0.0154, Train Acc: 99.40%, Dev Loss: 4.1662, Dev Acc: 45.60%
Epoch 4: Train Loss: 0.0145, Train Acc: 99.49%, Dev Loss: 4.1246, Dev Acc: 47.40%
Epoch 5: Train Loss: 0.0048, Train Acc: 99.62%, Dev Loss: 4.8214, Dev Acc: 47.40%
Epoch 6: Train Loss: 0.0040, Train Acc: 99.62%, Dev Loss: 4.9751, Dev Acc: 45.80%
Epoch 7: Train Loss: 0.0012, Train Acc: 99.71%, Dev Loss: 5.8263, Dev Acc: 46.60%
Epoch 8: Train Loss: 0.0014, Train Acc: 99.68%, Dev Loss: 6.1904, Dev Acc: 46.60%
Epoch 9: Train Loss: 0.0013, Train Acc: 99.68%, Dev Loss: 6.3965, Dev Acc: 46.80%
Epoch 10: Train Loss: 0.0017, Train Acc: 99.71%, Dev Loss: 6.1710, Dev Acc: 46.20%
Epoch 11: Train Loss: 0.0018, Train Acc: 99.67%, Dev Loss: 6.2412, Dev Acc: 46.80%
Epoch 12: Train Loss: 0.0013, Train Acc: 99.69%, Dev Loss: 6.5668, Dev Acc: 46.40%
Epoch 13: Tra

In [104]:
def predict(model, test_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            predictions.extend(pred.cpu().numpy().tolist())

    return [p[0] for p in predictions]

# Use the predict function to predict the labels of test data
predictions_test = predict(model, test_loader, device)

results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_test, predictions_test, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.490757,0.494038,0.447597


In [105]:
# Use the predict function to predict the labels of dev data
predictions_dev = predict(model, dev_loader, device)

results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, predictions_dev, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.473877,0.482541,0.437368


# SVM: Default params

In [ ]:
train_X = [np.array(x.cpu()).flatten() for x in train_img]
dev_X = [np.array(x.cpu()).flatten() for x in dev_img]
test_X = [np.array(x.cpu()).flatten() for x in test_img]

In [8]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='rbf') # parameter C was selected based on grid search
clf_svc.fit(train_X, Y_train)
Y_pred = clf_svc.predict(dev_X)
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.518329,0.502408,0.366971


In [9]:
Y_pred2 = clf_svc.predict(test_X)

In [10]:
df_dev = pd.read_csv(dev, keep_default_na=False)
df_test = pd.read_csv(test, keep_default_na=False)

df_dev['ResNet_svm_rbf_kernel'] = Y_pred
df_test['ResNet_svm_rbf_kernel'] = Y_pred2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

# SVM: C10

In [11]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='rbf', C=10) # parameter C was selected based on grid search
clf_svc.fit(train_X, Y_train)
Y_pred = clf_svc.predict(dev_X)
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.503555,0.501872,0.437037


In [12]:
Y_pred2 = clf_svc.predict(test_X)

In [13]:
# df_dev = pd.read_csv(dev, keep_default_na=False)
# df_test = pd.read_csv(test, keep_default_na=False)

df_dev['ResNet_svm_rbf_kernelC10'] = Y_pred
df_test['ResNet_svm_rbf_kernelC10'] = Y_pred2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

# SVM: Linear Kernel

In [14]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='linear') # parameter C was selected based on grid search
clf_svc.fit(train_X, Y_train)
Y_pred = clf_svc.predict(dev_X)
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.506658,0.505529,0.484847


In [15]:
Y_pred2 = clf_svc.predict(test_X)

In [16]:
# df_dev = pd.read_csv(dev, keep_default_na=False)
# df_test = pd.read_csv(test, keep_default_na=False)

df_dev['ResNet_svm_linear_kernel'] = Y_pred
df_test['ResNet_svm_linear_kernel'] = Y_pred2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

# SVM: Linear and C10

In [17]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='linear', C=10) # parameter C was selected based on grid search
clf_svc.fit(train_X, Y_train)
Y_pred = clf_svc.predict(dev_X)
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.504545,0.504081,0.492002


In [19]:
# df_dev = pd.read_csv(dev, keep_default_na=False)
# df_test = pd.read_csv(test, keep_default_na=False)
# Y_pred2 = clf_svc.predict(test_X)

df_dev['ResNet_svm_linear_kernelC10'] = Y_pred
df_test['ResNet_svm_linear_kernelC10'] = Y_pred2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)